<a href="https://colab.research.google.com/gist/hashikami/dde9e0b4737fef151835a623b4b72a18/car-pooling-problem-form-workspace.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Getting started**

1. Install Google OR-Tools.
2. Create small data model or large data model.
3. Run car pooling problem form workspace.

In [ ]:
!pip install ortools

In [ ]:
import itertools
import logging
import re

import numpy as np
from ortools.linear_solver import pywraplp


class FeasiblePath:
    """FeasiblePath
    Define feasible　path.

    Attributes:
        name (string): variable name z_p, p in P
        path (list): order list of clients picked up by driver
        distance (number): total distance
        duration (number): total duration
    """
    def __init__(self, name, path, distance, duration):
        self.name = name
        self.path = path
        self.distance = distance
        self.duration = duration


class CPPToWorkBySP:
    """CPPToWorkBySP
    Car pooling problem to workspace by by set partitioning.

    Attributes:
        data (dict):
            Vs (list): servers
            Vc (list): clients
            Q (list): capacities
            T (list): maximum ride times
            Ls (list): latest arrival time of servers
            Lc (list): latest arrival time of clients
            distance (2-d list): distance for each server and client
            duration (2-d list): duration for each server and client
    """
    def __init__(self, data):
        self.d = self.__format_data(data)
        self.feasible_paths = []
        self.cumsum_of_paths = [0]

    def __format_data(self, data):
        """ __format_data

        Format data model.
        """
        distance = []
        for vs_d in data['distance'][0:len(data['Vs'])]:
            distance.append(vs_d)
            distance.extend(data['distance'][len(data['Vs']):])
        data['distance'] = distance

        duration = []
        for vs_d in data['duration'][0:len(data['Vs'])]:
            duration.append(vs_d)
            duration.extend(data['duration'][len(data['Vs']):])
        data['duration'] = duration

        duration_vc_ti0 = []
        for vc in range(len(data['Vc'])):
            duration_vc_ti0.append(data['duration'][vc + 1][0])
        data['duration_vc_ti0'] = duration_vc_ti0

        l_vc = []
        for vs in range(len(data['Vs'])):
            l = []
            l.append(data['Ls'][vs])
            l.extend(data['Lc'])
            l_vc.append(l)
        data['L'] = l_vc

        return data

    def __create_paths(self, k):
        """__create_paths

        Create paths satisfying with capacity.

        Attributes:
            k (number): server index in Vs

        Returns:
            paths (list): paths (order list of clients)
            path_indexes (list): path indexes
        """
        paths = []
        path_indexes = []
        for Q in range(1, self.d['Q'][k]):
            vc = self.d['Vc']
            paths.extend(list(itertools.permutations(vc, Q)))

            vc_indexes = list(range(len(self.d['Vc'])))
            path_indexes.extend(list(itertools.permutations(vc_indexes, Q)))

        return paths, path_indexes

    def __calculate_cost(self, k, path_index):
        """__calculate_cost

        Calculate cost (total distance and duration) of path.

        Attributes:
            k (number): server index in 2D
            path_index (list): path index

        Returns:
            distance (number): total distance
            duration (number): total duration
        """
        workspace = 0

        # plus distance/duration between server and first client of path.
        first_vc = path_index[0]
        distance = self.d['distance'][k][first_vc + 1]
        duration = self.d['duration'][k][first_vc + 1]
        for vc, i in enumerate(path_index):
            # plus distance/duration between the client and workspace
            # if last client of path.
            if vc == len(path_index) - 1:
                distance += self.d['distance'][k + i + 1][workspace]
                duration += self.d['duration'][k + i + 1][workspace]
            # plus distance/duration between each client.
            else:
                next_vc = path_index[vc + 1]
                distance += self.d['distance'][k + i + 1][next_vc + 1]
                duration += self.d['duration'][k + i + 1][next_vc + 1]
        return distance, duration

    def __is_over_max_ride_time(self, k, duration):
        """__is_over_max_ride_time

        Cull paths over maximum ride time.

        Attributes:
            k (number): server index in Vs
            duration (number): total duration of path

        Returns:
            (boolean): over maximum ride time?
        """
        return duration > self.d['T'][k]

    def __is_out_of_arrival_time(self, k):
        """__is_out_of_arrival_time

        Cull paths out of arrival time.

        Attributes:
            k (number): server index in Vs

        Returns:
            (boolean): out of arrival time?
        """
        return False

    def __is_near_accident_spot(self):
        """__is_near_accident_spot

        Cull paths near accident blackspot

        Attributes:

        Returns:
            (boolean): near accident blackspot?
        """
        # TODO 事故多発地点を通るルートを削除する
        return False

    def __listup_feasible_paths(self):
        """__listup_feasible_paths

        List up feasible paths satisfying with capacity,
        maximum ride time, etc.
        """
        n = len(self.d['Vc']) + 1
        num_of_paths = [0]

        for k, vs in enumerate(self.d['Vs']):
            num_of_path = len(self.feasible_paths)

            # case of no client to pick up
            self.feasible_paths.append(
                FeasiblePath(
                    vs + ''.join(str(())),
                    (),
                    self.d['distance'][k * n][0],
                    self.d['duration'][k * n][0]))

            paths, path_indexes = self.__create_paths(k)
            for i, path in enumerate(paths):
                distance, duration = self.__calculate_cost(
                    k * n, path_indexes[i])

                if self.__is_over_max_ride_time(k, duration):
                    continue
                if self.__is_out_of_arrival_time(k):
                    continue
                if self.__is_near_accident_spot():
                    continue

                self.feasible_paths.append(FeasiblePath(
                    vs + ''.join(str(path)),
                    path,
                    distance,
                    duration))
            num_of_paths.append(len(self.feasible_paths) - num_of_path)
        self.cumsum_of_paths = np.cumsum(num_of_paths)

    def __create_variables(self, solver):
        """ __create_variables

        Create variables.

        Returns:
            z (list): z_p, p in P
            y (list): y_i, i in Vc
        """
        z = [solver.IntVar(0, 1, 'z({})'.format(p.name))
             for p in self.feasible_paths]
        y = [solver.IntVar(0, 1, 'y({})'.format(i)) for i in self.d['Vc']]
        return z, y

    def __create_constraints(self, solver, z, y):
        """ __create_constraints

        Create constraints.

        Attributes:
            z (list): z_p, p in P
            y (list): y_i, i in Vc
        """
        # Each server drives a path.
        for k, _ in enumerate(self.d['Vs']):
            solver.Add(solver.Sum(
                z[self.cumsum_of_paths[k]:self.cumsum_of_paths[k + 1]]) == 1)

        # Each client is either picked up by a server or is left unserviced.
        for i, vc in enumerate(self.d['Vc']):
            solver.Add(
                solver.Sum([z[p] for p, fp in enumerate(self.feasible_paths)
                            if vc in fp.path]) + y[i] == 1)

    def __create_objective(self, solver, z, y):
        """ __create_objective

        Create objective.

        Attributes:
            z (list): z_p, p in P
            y (list): y_i, i in Vc
        """
        solver.Minimize(
            solver.Sum([np.inner(p.duration, z_p)
                        for p, z_p in zip(self.feasible_paths, z)]) +
            solver.Sum([np.inner(1.0 * np.array(
                self.d['duration_vc_ti0']), y)]))

    def solve(self):
        """solve

        Solve CPP by set partitioning

        Returns:
            feasible_paths (dict): feasible paths
            %m (number): matching rate
            %tdr (number): total distance reduction rate
            %rti (number): ride time increase rat
        """
        self.__listup_feasible_paths()

        solver = pywraplp.Solver(
            'cp', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

        z, y = self.__create_variables(solver)
        self.__create_constraints(solver, z, y)
        self.__create_objective(solver, z, y)

        status = solver.Solve()

        paths = {}
        if status == pywraplp.Solver.OPTIMAL:
            root = []
            for p, _ in enumerate(z):
                if z[p].solution_value():
                    root.append(z[p].name())
                    r = re.findall(r'\w+', z[p].name())
                    paths[r[1]] = r[2:]
            not_pickup_clients = []
            for i in range(len(self.d['Vc'])):
                not_pickup_clients.append((y[i].name(), y[i].solution_value()))
            logging.info(
                'Root %s, and not pickup clients %s solved in %d variables, \
                %d constraints, %f objective value, %f milliseconds, \
                %d iterations, and %d branch-and-bound nodes.' % (
                    root, not_pickup_clients,
                    solver.NumVariables(),
                    solver.NumConstraints(),
                    solver.Objective().Value(),
                    solver.wall_time(), solver.iterations(), solver.nodes()))
        else:
            logging.info('The problem does not have an optimal solution.')

        m, tdr, rti = self.__evaluate(z, y)
        return {
            'feasible_paths': paths,
            '%m': m,
            '%tdr': tdr,
            '%rti': rti
        }

    def __evaluate(self, z, y):
        total_distance = 0
        total_duration = 0
        original_distance = 0
        original_duration_vs = 0
        for p in range(len(z)):
            if z[p].solution_value():
                total_distance += self.feasible_paths[p].distance
                total_duration += self.feasible_paths[p].duration
        n = len(self.d['Vc']) + 1
        for i in range(len(self.d['Vs'])):
            original_distance += self.d['distance'][i * n][0]
            original_duration_vs += self.d['duration'][i * n][0]
        not_pickup_clients = 0
        for i in range(len(self.d['Vc'])):
            if y[i].solution_value() == 1:
                not_pickup_clients += 1
                total_distance += self.d['distance'][i + 1][0]
            original_distance += self.d['distance'][i + 1][0]
        m = (len(self.d['Vc']) - not_pickup_clients) / len(self.d['Vc'])
        tdr = 1 - total_distance / original_distance
        rti = total_duration / original_duration_vs
        return m, tdr, rti

data = create_data_model()
output = CPPToWorkBySP(data).solve()
print(output)

{'feasible_paths': {'3': ['264', '282', '242'], '5': ['301', '168', '274'], '13': ['229', '160', '248'], '14': [], '17': ['235', '190', '205'], '26': ['222', '157'], '29': ['245'], '40': ['196', '208', '249', '237'], '41': [], '44': ['177']}, '%m': 1.0, '%tdr': 0.37575856121625306, '%rti': 1.6537704918032787}


In [ ]:
def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data['Vs'] = ['a', 'b']
    data['Vc'] = ['c', 'd', 'e', 'f', 'g', 'h', 'i']
    data['Q'] = [7, 5]
    data['T'] = [12, 15]
    data['Ls'] = [120, 120]
    data['Lc'] = [60, 60, 60, 60, 60, 60, 101]
    data['distance'] = [
                        [10, 5, 7, 10, 1, 3, 10, 100],
                        [10, 7, 9, 4, 11, 11, 12, 100],
                        [5, 0, 2, 5, 4, 2, 5, 100],
                        [3, 2, 0, 5, 6, 4, 3, 100],
                        [6, 5, 5, 0, 9, 7, 8, 100],
                        [9, 4, 6, 9, 0, 2, 9, 100],
                        [7, 2, 4, 7, 2, 0, 7, 100],
                        [1, 5, 3, 8, 9, 7, 0, 100],
                        [100, 100, 100, 100, 100, 100, 100, 0],
                        ]
    data['duration'] = [
                        [10, 5, 7, 10, 1, 3, 10, 100],
                        [10, 7, 9, 4, 11, 11, 12, 100],
                        [5, 0, 2, 5, 4, 2, 5, 100],
                        [3, 2, 0, 5, 6, 4, 3, 100],
                        [6, 5, 5, 0, 9, 7, 8, 100],
                        [9, 4, 6, 9, 0, 2, 9, 100],
                        [7, 2, 4, 7, 2, 0, 7, 100],
                        [1, 5, 3, 8, 9, 7, 0, 100],
                        [100, 100, 100, 100, 100, 100, 100, 0],
                        ]
    return data

In [ ]:
def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data['Vs'] = ['3', '5', '13', '14', '17', '26', '29', '40', '41', '44']
    data['Vc'] = ['157', '160', '168', '177', '190', '196', '205', '208', '222', '229', '235', '237', '242', '245', '248', '249', '264', '274', '282', '301']
    data['Q'] = [5, 5, 5, 5, 5, 5, 5, 5, 5, 5]
    data['T'] = [3600, 3600, 3600, 3600, 3600, 3600, 3600, 3600, 3600, 3600]
    data['Ls'] = [7200, 7200, 7200, 7200, 7200, 7200, 7200, 7200, 7200, 7200]
    data['Lc'] = [7200, 7200, 7200, 7200, 7200, 7200, 7200, 7200, 7200, 7200, 7200, 7200, 7200, 7200, 7200, 7200, 7200, 7200, 7200, 7200]
    data['distance'] = [[5945,5095,8433,4113,4080,2195,3817,408,7770,4305,7509,3146,5903,1784,7159,8433,8189,4936,6153,4225,2625],
                        [6536,3977,9076,3358,4401,2318,4408,922,7015,6567,8670,3467,5148,2501,7750,9076,8432,5191,5398,4546,2880],
                        [6043,12573,1788,6739,7149,7045,6547,8618,4219,10344,3351,7244,4181,7374,3071,1788,2665,5704,5164,6714,6247],
                        [3824,8453,6311,4339,1418,2930,1696,4432,7132,2075,5388,1015,5287,2220,5038,6311,7491,3496,6379,2725,2672],
                        [8022,3780,9733,4137,5058,3183,5894,2175,7672,7224,9327,4124,5805,3366,9047,9733,9089,5848,5864,5203,3537],
                        [8202,5281,10153,7553,5940,4577,5537,3749,10147,5547,10224,5356,8302,3693,9874,10153,10506,7253,9593,6303,5547],
                        [7623,9725,5969,4522,5424,5320,6819,6808,1588,8619,6047,5519,1454,5649,5697,5969,3005,3979,2553,4989,3822],
                        [7209,4661,8665,2947,3990,2412,5777,1815,6604,6156,8259,3056,4737,2717,7979,8665,8021,4780,4987,4135,2469],
                        [4129,7785,5773,3671,605,2262,2765,3764,6008,3339,5367,978,4163,2590,4891,5773,6367,1601,5711,1060,1392],
                        [6007,13056,1752,7222,7632,7528,6511,9101,4702,8677,3315,7727,4664,7857,3035,1752,3148,6187,5647,7197,6730],
                        [11788,0,23024,7029,8071,6494,9123,5230,10686,9133,13810,7137,8819,7279,13460,23024,12103,8862,9069,8216,6551],
                        [4727,23024,0,7096,5859,7509,5231,9082,5681,7397,2035,6422,5315,7738,1755,0,4127,4609,6626,5206,6711],
                        [5698,7029,7096,0,4224,3460,4573,4074,4875,6390,6526,3290,2686,3915,6246,7311,6292,3075,2320,3007,1694],
                        [4067,8071,5859,4224,0,2236,1939,4344,6311,2994,4747,1558,4466,3334,4397,5706,6670,1534,5501,993,1682],
                        [4988,6494,7509,3460,2236,0,3185,2377,5738,4148,6425,1048,3893,1154,6145,6831,6097,2844,4957,1894,1138],
                        [2238,9123,5231,4573,1939,3185,0,4201,7534,2766,4260,2691,5689,2931,3910,5039,7222,2758,7017,2169,2906],
                        [6199,5230,9082,4074,4344,2377,4201,0,7634,4559,7763,3010,5767,1986,7413,8499,7765,4512,6017,4089,2489],
                        [8812,10686,5681,4875,6311,5738,7534,7634,0,9485,6760,6385,2320,6515,6410,6184,1417,4845,3304,5855,4688],
                        [4177,9133,7397,6390,2994,4148,2766,4559,9485,0,6199,3093,7365,3429,5849,7205,9569,4440,8457,3899,4750],
                        [3375,13810,2035,6526,4747,6425,4260,7763,6760,6199,0,6789,5050,7086,350,2890,5189,3872,6753,5958,6694],
                        [4842,7137,6422,3290,1558,1048,2691,3010,6385,3093,6789,0,4264,2236,5547,6233,6468,2474,5357,1703,1649],
                        [6595,8819,5315,2686,4466,3893,5689,5767,2320,7365,5050,4264,0,4621,4669,5664,3592,2951,1699,3961,3353],
                        [5760,7279,7738,3915,3334,1154,2931,1986,6515,3429,7086,2236,4621,0,6916,7602,6868,3615,5729,2665,1910],
                        [3025,13460,1755,6246,4397,6145,3910,7413,6410,5849,350,5547,4669,6916,0,2540,4839,3522,6403,5608,6344],
                        [4727,23024,0,7311,5706,6831,5039,8499,6184,7205,2890,6233,5664,7602,2540,0,4127,4609,6626,5206,6711],
                        [7118,12103,4127,6292,6670,6097,7222,7765,1417,9569,5189,6468,3592,6868,4839,4127,0,5067,4721,6077,5610],
                        [3956,8862,4609,3075,1534,2844,2758,4512,4845,4440,3872,2474,2951,3615,3522,4609,5067,0,4394,863,1966],
                        [8094,9069,6626,2320,5501,4957,7017,6017,3304,8457,6753,5357,1699,5729,6403,6626,4721,4394,0,5460,3760],
                        [3783,8216,5206,3007,993,1894,2169,4089,5855,3899,5958,1703,3961,2665,5608,5206,6077,863,5460,0,1608],
                        [4692,6551,6711,1694,1682,1138,2906,2489,4688,4750,6694,1649,3353,1910,6344,6711,5610,1966,3760,1608,0]]
    data['duration'] = [[1027,707,1332,686,582,374,602,132,1157,605,1239,387,966,315,1149,1332,1208,784,960,667,457],
                        [1130,539,1349,537,541,362,705,216,1008,646,1250,346,817,349,1252,1349,1150,784,811,626,458],
                        [1113,1742,394,1045,1100,1039,1090,1303,570,1316,642,1016,652,1067,563,394,414,812,780,1041,963],
                        [701,934,1006,550,184,292,276,491,995,226,913,74,829,289,823,1006,993,538,824,399,341],
                        [1288,565,1467,632,659,499,863,406,1125,764,1368,464,934,486,1289,1467,1267,902,916,744,576],
                        [1049,496,1447,1011,746,531,717,513,1346,658,1294,571,1180,445,1204,1447,1343,919,1285,862,712],
                        [1311,1433,839,681,983,922,1127,1113,247,1199,1059,899,304,950,969,839,388,695,379,924,812],
                        [1126,660,1342,530,534,389,688,379,1001,639,1243,339,810,396,1164,1342,1143,778,804,619,452],
                        [804,981,1044,597,144,340,457,538,1029,379,945,170,863,368,898,1044,1027,423,871,291,385],
                        [1126,1843,407,1146,1201,1140,1103,1404,671,1414,655,1117,753,1168,576,407,515,913,881,1142,1064],
                        [1462,0,1520,1043,1047,902,1138,699,1514,1079,1707,852,1323,866,1617,1520,1656,1291,1317,1132,965],
                        [868,1520,0,1134,1007,1151,845,1415,818,1156,397,1060,920,1207,318,0,662,692,1028,852,1075],
                        [1184,1043,1134,0,679,566,882,704,725,784,1093,484,535,627,1014,1122,867,649,485,725,453],
                        [725,1047,1007,679,0,415,300,629,1039,416,915,261,873,475,825,1013,1037,392,979,260,387],
                        [855,902,1151,566,415,0,507,459,860,415,941,115,694,214,862,1040,857,433,819,376,227],
                        [486,1138,845,882,300,507,0,727,1045,358,731,363,879,471,641,824,1011,501,1039,371,487],
                        [1154,699,1415,704,629,459,727,0,1216,732,1366,445,1025,327,1276,1336,1154,729,1019,725,515],
                        [1284,1514,818,725,1039,860,1045,1216,0,1228,985,928,333,979,895,772,142,724,463,953,841],
                        [755,1079,1156,784,416,415,358,732,1228,0,1000,308,1062,461,910,1124,1226,769,1057,637,575],
                        [623,1707,397,1093,915,941,731,1366,985,1000,0,958,844,1064,74,527,753,612,1067,875,991],
                        [774,852,1060,484,261,115,363,445,928,308,958,0,754,246,869,1047,918,463,749,324,267],
                        [1150,1323,920,535,873,694,879,1025,333,1062,844,754,0,789,803,778,443,534,304,763,654],
                        [936,866,1207,627,475,214,471,327,979,461,1064,246,789,0,943,1121,939,514,900,458,308],
                        [548,1617,318,1014,825,862,641,1276,895,910,74,869,803,943,0,452,678,538,992,800,916],
                        [868,1520,0,1122,1013,1040,824,1336,772,1124,527,1047,778,1121,452,0,662,692,1028,852,1075],
                        [1097,1656,662,867,1037,857,1011,1154,142,1226,753,918,443,939,678,662,0,674,616,903,825],
                        [691,1291,692,649,392,433,501,729,724,769,612,463,534,514,538,692,674,0,725,213,472],
                        [1382,1317,1028,485,979,819,1039,1019,463,1057,1067,749,304,900,992,1028,616,725,0,995,749],
                        [737,1132,852,725,260,376,371,725,953,637,875,324,763,458,800,852,903,213,995,0,377],
                        [856,965,1075,453,387,227,487,515,841,575,991,267,654,308,916,1075,825,472,749,377,0]]
    return data